In [1]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt 
import numpy as np
import os
import pandas as pd 

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

/Users/dweepa/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data = pd.read_csv("../Dataset/afterPCA.csv",low_memory=False)
data.drop(['Unnamed: 0'],axis=1, inplace=True)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,case_status,class_of_admission
0,1.718837,-1.564101,-0.624270,0.367709,0.787634,0.745779,-0.765707,-1.596453,-1.157607,0.113230,...,-0.922478,0.366428,4.181433,0.626535,-2.943687,0.965737,0.967921,-0.684650,1,0
1,-3.220177,-2.130617,0.059023,1.123626,-1.581496,-0.053500,-0.764700,0.048586,0.543452,0.215647,...,-0.082052,0.325926,0.313514,-0.302003,0.611517,-0.233497,-0.741353,0.197093,0,0
2,-0.683024,-2.358398,-0.380102,0.324592,1.533251,1.329765,-0.661416,0.501212,-0.193716,2.081190,...,-0.344387,0.231931,-0.131912,0.825182,-0.686843,-0.775654,0.987073,0.298712,1,0
3,-0.892537,-1.527530,1.762655,-2.958144,2.314363,1.380884,0.675292,-1.719787,1.551155,-1.631679,...,-1.154058,-1.841991,0.730057,-2.563708,-0.313658,-0.725614,-3.706364,-0.036169,1,0
4,1.378325,0.700173,1.513348,0.407777,1.708383,0.722666,-1.202219,1.425830,-0.149194,0.524412,...,0.225081,0.571899,-0.820183,0.676044,-0.288469,-0.799120,0.177217,0.058635,1,0


### Making one hot encoding into a proper list format and checking if classifiers can take it
#### (Did not work)


### def makelist(d):

    print(len(d))
    for i in range(len(d)):
        a= d[i]
        a=a[1:len(a)-1]
        a=a.split(', ')
        a=[int(float(i)) for i in a]
        d[i]=a
        
## def makelist1(a):

        #print(type(a))
        a=a[1:len(a)-1]
        a=a.split(', ')
        l=[int(float(i)) for i in a]
        #print(type(l))
        return l

## Splitting into testing and training

In [3]:
y=data['case_status']
data1=data.drop(['case_status',],axis=1)
X=data1
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, test_size=0.2)

In [4]:
a = [i for i in y_train1 if i==0]
len(a)
len(y_train1)-len(a)

116647

## Neural Networks

In [5]:
classifier = Sequential()
classifier.add(Dense(activation="relu", input_dim=X_train1.shape[1], units=6, kernel_initializer="uniform"))
classifier.add(Dense(activation="relu", units=10, kernel_initializer="uniform"))
classifier.add(Dense(activation="relu", units=10, kernel_initializer="uniform"))
classifier.add(Dense(activation="relu", units=10, kernel_initializer="uniform"))
classifier.add(Dropout(0.5))
classifier.add(Dense(activation="relu", units=1, kernel_initializer="uniform"))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.fit(X_train1, y_train1, batch_size = 100, epochs = 30,verbose=0)

In [6]:
score= classifier.evaluate(X_test1,y_test1)
print(score)

30155/30155 [==============================] - 1s 20us/step
[0.31022538434725816, 0.671629912118733]


In [7]:
y_pred=classifier.predict(X_test1)
print("Neural Network:",accuracy_score(y_pred.round(),y_test1))

Neural Network: 0.6716299121207097


### predict everything as 1

In [8]:
l=len([i for i in y_pred.round() if i==0])
l

0

## Other Models

In [9]:
seed = 7
# prepare models
models = []
models.append(('Logistic Regression', LogisticRegression()))
models.append(('Linear Discriminant Analysis', LinearDiscriminantAnalysis()))
models.append(('K Nearest Neighbours', KNeighborsClassifier()))
models.append(('Decision Trees', DecisionTreeClassifier()))
models.append(('Naive Bayes', GaussianNB()))
#models.append(('Support Vector Machine', SVC()))
# evaluate each model in turn
results = []
names = []
scoring = 'accuracy'

In [10]:
modellist=[['Model','Accuracy','#DeniedPredictions']]
for name, model in models:
    try:
        model.fit(X_train1,y_train1)
        y_test1.values.reshape(-1,1)
        y_pred = model.predict(X_test1)
        #print(y_pred)
        l=(len([i for i in y_pred.round() if i==0]))
        cv_results = model.score(X_test1,y_test1)
        results.append(cv_results)
        names.append(name)
        msg = [name, cv_results,l]
        modellist.append(msg)
        print(msg)
    except:
        print(name)


['Logistic Regression', 0.9651135798375062, 2]
['Linear Discriminant Analysis', 0.9637539379870668, 121]
['K Nearest Neighbours', 0.966174763720776, 202]
['Decision Trees', 0.9416680484165146, 1145]
['Naive Bayes', 0.40620129331785776, 18502]


In [11]:
modellist.append(['Neural Networks',accuracy_score(y_pred.round(),y_test1),l])
                  f = pd.DataFrame(modellist)
df

IndentationError: unexpected indent (<ipython-input-11-6abdc73eeaa0>, line 2)